In [1]:
#we load our libraries
import pandas as pd
import sys
sys.path.append('/Users/halukamb/Documents/CodeRelated/HBDataviz_new/2025/01_BTW100Deutsche/Wrangle/lib/python3.12/site-packages')
import numpy as np
from json import loads, dumps

In [2]:
! pip install pyreadstat


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
#we have one function that filters the ALLBUS data for whether respondents are german and of voting age
def germanElectorate(df):
    german_df=df[df["german"]=="JA"]
    german_df=german_df.dropna(subset="age")
    german_df["age"]=german_df["age"].astype(float)
    german_electorate=german_df[german_df["age"]>=18]
    return german_electorate

In [4]:
#we have another function that regroups the data into fitting categories to the data we have from the zensus
def reformgroups(df):
    df["Alter"]=None
    df["Alter"]=np.where(df["age"]<30,"U30",df["Alter"])
    df["Alter"]=np.where(df["age"]>=30,"30-49",df["Alter"])
    df["Alter"]=np.where(df["age"]>=50,"50-64",df["Alter"])
    df["Alter"]=np.where(df["age"]>64,"Ü65",df["Alter"])
    df["Geschlecht"]=np.nan
    df["Geschlecht"]=np.where(df["sex"]=="FRAU","Weiblich",df["Geschlecht"])
    df["Geschlecht"]=np.where(df["sex"]=="MANN","Männlich",df["Geschlecht"])
    df["OstWest"]=np.nan
    df["OstWest"]=np.where(df["eastwest"]=="ALTE BUNDESLAENDER","West",df["OstWest"])
    df["OstWest"]=np.where(df["eastwest"]=="NEUE BUNDESLAENDER","Ost",df["OstWest"])
    df["OstWest"]=np.where(df["land"].str.contains("BERLIN"),"Berlin",df["OstWest"])

    df["Besiedelung"]=np.nan
    df["Besiedelung"]=np.where(df["gs01"].isin(['GROSSSTADT']),"dicht besiedelt",df["Besiedelung"])
    df["Besiedelung"]=np.where(df["gs01"].isin( ['MITTEL-, KLEINSTADT', 'VORORT GROSSSTADT']),"mittlere Besiedlungsdichte",df["Besiedelung"])
    df["Besiedelung"]=np.where(df["gs01"].isin( ['LAENDL. DORF','EINZELHAUS, LAND']),"gering besiedelt",df["Besiedelung"])
    
    


    
    return df

In [5]:
#and lastly we have a stratification function that takes the zensus data and the skewed dataset of allbus
#  and creates a stratified dataset based on the population shares of the zensus data
#for each combination of age, gender, eastwest and population density
def stratifier(zensus_model,skewed_df):

    stratified_dataset=pd.DataFrame()
    samplesize_sum=0

    #we go through each row of the zensus groups
    for index, row in zensus_model.iterrows():
        #take in which share of the population this group represents
        samplesize = round(row["Share"] * 100)
        samplesize_sum += samplesize
        #and then we take a copy of the skewed allbus data
        sub_df = skewed_df.copy()
        
        for i in range(0, 4):
            #and then we filter it down, feature by feature of the speficic row of the zensus data
            #we do this from 0 to 4 because in the first four rows we have the features we want to stratify by
            sub_df = sub_df[sub_df[row.index[i]] == row.values[i]]
            print(row.index[i],row.values[i],len(sub_df))
        
        
        #we draw a sample from this group that correlates to the share of the population
        #if the original dataset is too small, we sample with replacement
        sampled_subgroup = sub_df.sample(samplesize, random_state=1, replace=True)
        #and add this sampled group to the overall stratified dataset
        stratified_dataset = pd.concat([stratified_dataset, sampled_subgroup])
    print(len(stratified_dataset),samplesize_sum)
    return stratified_dataset


In [6]:
#this is the zensus data we need, we remove all rows with NaN values
#as we need as detailed data as possible for the stratification
zensus_model=pd.read_csv("zensus_model.csv")
zensus_model=zensus_model.dropna()
zensus_model=zensus_model.sort_values(by="Share",ascending=False)


In [7]:
#we read in the allbus data for 2021
df_2021 = pd.read_spss("../data/AllbusData/ZA5280_v2-0-1.sav")
# filter it for the german electorate

df_2021 = germanElectorate(df_2021)
# regroup it to fit the zensus data

df_2021= reformgroups(df_2021)
# and stratify it based on the zensus data
df_2021= stratifier(zensus_model,df_2021)
df_2021

Geschlecht Weiblich 2474
Besiedelung mittlere Besiedlungsdichte 1204
OstWest West 744
Alter Ü65 223
Geschlecht Weiblich 2474
Besiedelung mittlere Besiedlungsdichte 1204
OstWest West 744
Alter 50-64 247
Geschlecht Männlich 2419
Besiedelung mittlere Besiedlungsdichte 1182
OstWest West 752
Alter 50-64 233
Geschlecht Weiblich 2474
Besiedelung mittlere Besiedlungsdichte 1204
OstWest West 744
Alter 30-49 193
Geschlecht Männlich 2419
Besiedelung mittlere Besiedlungsdichte 1182
OstWest West 752
Alter 30-49 176
Geschlecht Männlich 2419
Besiedelung mittlere Besiedlungsdichte 1182
OstWest West 752
Alter Ü65 263
Geschlecht Weiblich 2474
Besiedelung dicht besiedelt 465
OstWest West 250
Alter Ü65 60
Geschlecht Männlich 2419
Besiedelung dicht besiedelt 451
OstWest West 247
Alter 30-49 79
Geschlecht Weiblich 2474
Besiedelung dicht besiedelt 465
OstWest West 250
Alter 30-49 83
Geschlecht Weiblich 2474
Besiedelung dicht besiedelt 465
OstWest West 250
Alter 50-64 60
Geschlecht Männlich 2419
Besiedelung d

,za_nr,doi,version,respid,substudy,mode,splt21,eastwest,german,ep01,...,bik,gkpol,wghtpew,wghtht,wghthew,wghthtew,Alter,Geschlecht,OstWest,Besiedelung
986,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),987.0,SIMULTAN,MAIL,SPLIT A,ALTE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 1 ;>499.999,100.000 - 499.999 E.,ALTE BUNDESLAENDER,1.685751,ALTE BUNDESLAENDER,2.083416,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3459,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),3460.0,SIMULTAN,MAIL,SPLIT C,ALTE BUNDESLAENDER,JA,NaN,...,ZONE 1 ;<500.000,100.000 - 499.999 E.,ALTE BUNDESLAENDER,NaN,NaN,NaN,Ü65,Weiblich,West,mittlere Besiedlungsdichte
1790,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),1791.0,SIMULTAN,MAIL,SPLIT A,ALTE BUNDESLAENDER,JA,GUT,...,ZONE 2-4;>499.999,20.000 - 49.999 EINW,ALTE BUNDESLAENDER,0.842875,ALTE BUNDESLAENDER,1.041708,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3381,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),3382.0,SIMULTAN,MAIL,SPLIT A,ALTE BUNDESLAENDER,JA,NaN,...,ZONE 1 ;>499.999,500.000 UND MEHR E.,ALTE BUNDESLAENDER,1.685751,ALTE BUNDESLAENDER,2.083416,Ü65,Weiblich,West,mittlere Besiedlungsdichte
4800,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4801.0,SEQUENZIELL,CAWI,SPLIT C,ALTE BUNDESLAENDER,JA,NaN,...,ZONE 2-4;<500.000,5.000 - 19.999 EINW,ALTE BUNDESLAENDER,0.842875,ALTE BUNDESLAENDER,1.041708,Ü65,Weiblich,West,mittlere Besiedlungsdichte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4710,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4711.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 2-4;<100.000,2.000 - 4.999 EINW,NEUE BUNDESLAENDER,0.267321,NEUE BUNDESLAENDER,0.141565,U30,Weiblich,Ost,gering besiedelt
1464,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),1465.0,SIMULTAN,MAIL,SPLIT B,NEUE BUNDESLAENDER,JA,SCHLECHT,...,ZONE 2-4;<500.000,20.000 - 49.999 EINW,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,U30,Weiblich,Ost,gering besiedelt
4710,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4711.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 2-4;<100.000,2.000 - 4.999 EINW,NEUE BUNDESLAENDER,0.267321,NEUE BUNDESLAENDER,0.141565,U30,Weiblich,Ost,gering besiedelt
2291,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),2292.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 2-4;>499.999,5.000 - 19.999 EINW,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,U30,Weiblich,Ost,gering besiedelt


In [8]:
#we read in the allbus data for 2023
df_2023 = pd.read_spss("../data/AllbusData/ZA8830_v1-0-0.sav")
# filter it for the german electorate

df_2023 = germanElectorate(df_2023)
# regroup it to fit the zensus data

df_2023= reformgroups(df_2023)
#and straitify it based on the zensus data
df_2023= stratifier(zensus_model,df_2023)
df_2023


Geschlecht Weiblich 2412
Besiedelung mittlere Besiedlungsdichte 1224
OstWest West 729
Alter Ü65 208
Geschlecht Weiblich 2412
Besiedelung mittlere Besiedlungsdichte 1224
OstWest West 729
Alter 50-64 230
Geschlecht Männlich 2292
Besiedelung mittlere Besiedlungsdichte 1078
OstWest West 666
Alter 50-64 184
Geschlecht Weiblich 2412
Besiedelung mittlere Besiedlungsdichte 1224
OstWest West 729
Alter 30-49 197
Geschlecht Männlich 2292
Besiedelung mittlere Besiedlungsdichte 1078
OstWest West 666
Alter 30-49 178
Geschlecht Männlich 2292
Besiedelung mittlere Besiedlungsdichte 1078
OstWest West 666
Alter Ü65 222
Geschlecht Weiblich 2412
Besiedelung dicht besiedelt 485
OstWest West 259
Alter Ü65 76
Geschlecht Männlich 2292
Besiedelung dicht besiedelt 471
OstWest West 257
Alter 30-49 85
Geschlecht Weiblich 2412
Besiedelung dicht besiedelt 485
OstWest West 259
Alter 30-49 60
Geschlecht Weiblich 2412
Besiedelung dicht besiedelt 485
OstWest West 259
Alter 50-64 64
Geschlecht Männlich 2292
Besiedelung d

,za_nr,doi,version,respid,substudy,mode,splt23_1,splt23_2,eastwest,german,...,bik,xs11,wghtpew,wghtht,wghthew,wghthtew,Alter,Geschlecht,OstWest,Besiedelung
770,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",771.0,NaN,CAPI,NaN,SPLIT A,ALTE BUNDESLAENDER,JA,...,ZONE 2-4;>499.999,183.0,1.252326,1.69639,1.242308,2.107439,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3538,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",3539.0,NaN,CAPI,NaN,SPLIT B,ALTE BUNDESLAENDER,JA,...,ZONE 1 ;>499.999,93.0,1.252326,0.848195,1.242308,1.053719,Ü65,Weiblich,West,mittlere Besiedlungsdichte
1776,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",1777.0,SIMULTAN,MAIL,SPLIT B,TNZ: MODE,ALTE BUNDESLAENDER,JA,...,ZONE 1-4;<50.000,10.0,1.203727,1.693259,1.192166,2.018646,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3432,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",3433.0,SEQUENZIELL,CAWI,SPLIT A,TNZ: MODE,ALTE BUNDESLAENDER,JA,...,ZONE 1 ;<500.000,168.0,1.207467,0.554394,1.205184,0.668147,Ü65,Weiblich,West,mittlere Besiedlungsdichte
5150,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",5151.0,NaN,CAPI,NaN,SPLIT B,ALTE BUNDESLAENDER,JA,...,ZONE 2-4;>499.999,143.0,1.252326,0.848195,1.242308,1.053719,Ü65,Weiblich,West,mittlere Besiedlungsdichte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1897,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",1898.0,NaN,CAPI,NaN,SPLIT B,NEUE BUNDESLAENDER,JA,...,5.000-19.999 EINW.,55.0,0.501346,0.532744,0.524886,0.27963,U30,Weiblich,Ost,gering besiedelt
3272,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",3273.0,SIMULTAN,MAIL,SPLIT A,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 1 ;<500.000,176.0,0.544816,0.845091,0.572064,0.483446,U30,Weiblich,Ost,gering besiedelt
4876,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",4877.0,SIMULTAN,MAIL,SPLIT A,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 2-4;<500.000,31.0,0.544816,0.338036,0.572064,0.193378,U30,Weiblich,Ost,gering besiedelt
4876,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",4877.0,SIMULTAN,MAIL,SPLIT A,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 2-4;<500.000,31.0,0.544816,0.338036,0.572064,0.193378,U30,Weiblich,Ost,gering besiedelt


In [9]:
#this is a function that enables us to analyze for each for each subgroup how the respondents answered differently
#  to a given question/variable
def analyze_factor_patterns(dataset,variable_to_check):
    #we have a base dataframe that holds all results
    base_df=pd.DataFrame()
    #now we define the four features we want to use to check for changes in the respondents data in allbus
    factor_list=["Alter","Geschlecht","OstWest","Besiedelung"]
    #we iterate through all possible combinations of these features
    for element in factor_list:
        #print(element)
        for element2 in factor_list:
            #add we za_nr to the list of the variable to check, as it is the unique identifier for each respondent
            baselist=[variable_to_check,"za_nr"]

            #we build also a list of the features we want to check for
            extralist=[element,element2]
            #set it to remove duplicates, in case we pick the same feature twice
            extralist=list(set(extralist))

            #and combine the two lists and filter the dataset for the columns we need
            baselist.extend(extralist)
            dataset_reduced=dataset[baselist]
            #remove all rows with missing values
            dataset_reduced = dataset_reduced[~dataset_reduced.isin(["nan"]).any(axis=1)]

            #we also define a extralist plus which we need to group by the dataset
            #it is the name of the features we want to check for plus the variable
            # and then we count
            extralist_plus_factor=extralist+[variable_to_check]
            grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
            #we remove all columns that have missing values
            grouped_df=grouped_df.dropna(axis=1)
            #and then we calculate the share of the respondents for each subgroup by using the sum of the group
            grouped_df['percent'] = grouped_df['za_nr']*100 / grouped_df.groupby(by=extralist)['za_nr'].transform('sum')
            grouped_df['absolute'] = grouped_df['za_nr']
            
            #and we add this to the base dataframe that holds all results
            grouped_df=grouped_df.reset_index()
            base_df=pd.concat([base_df,grouped_df],axis=0)
    #at the end we drop duplicates
    base_df=base_df.drop_duplicates()
    #reduce the dataset
    base_df_cols=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', variable_to_check,
       'absolute','percent']
    #and return the dataset
    return base_df[base_df_cols]

In [10]:
# now we do this whole procedure for variable va03 which is for inflation
factor_to_check="mc03"

df_2023_factor=analyze_factor_patterns(df_2023,factor_to_check)
df_2023_factor=df_2023_factor.sort_values(by="absolute",ascending=False)
df_2023_factor

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_59960/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_59960/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_59960/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,mc03,absolute,percent
4,NaN,NaN,NaN,West,JA,3279,60.755976
5,NaN,NaN,NaN,West,NEIN,2118,39.244024
0,NaN,NaN,Männlich,NaN,JA,1946,59.041262
2,NaN,NaN,Weiblich,NaN,JA,1853,52.716927
4,NaN,mittlere Besiedlungsdichte,NaN,NaN,JA,1747,56.610499
...,...,...,...,...,...,...,...
1,30-49,NaN,NaN,Berlin,NEIN,8,8.988764
12,NaN,mittlere Besiedlungsdichte,NaN,Berlin,JA,0,NaN
13,NaN,mittlere Besiedlungsdichte,NaN,Berlin,NEIN,0,NaN
7,NaN,gering besiedelt,NaN,Berlin,NEIN,0,NaN


In [11]:
# now we do this whole procedure for variable va03 which is for inflation

df_2021_factor=analyze_factor_patterns(df_2021,factor_to_check)
df_2021_factor=df_2021_factor.sort_values(by="absolute",ascending=False)
df_2021_factor

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_59960/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_59960/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_59960/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,mc03,absolute,percent
4,NaN,NaN,NaN,West,JA,3274,66.342452
2,NaN,NaN,Weiblich,NaN,JA,1917,58.767627
0,NaN,NaN,Männlich,NaN,JA,1819,61.933946
10,NaN,NaN,Weiblich,West,JA,1696,64.560335
5,NaN,NaN,NaN,West,NEIN,1661,33.657548
...,...,...,...,...,...,...,...
19,Ü65,NaN,NaN,Berlin,NEIN,16,32.653061
12,NaN,mittlere Besiedlungsdichte,NaN,Berlin,JA,0,NaN
13,NaN,mittlere Besiedlungsdichte,NaN,Berlin,NEIN,0,NaN
7,NaN,gering besiedelt,NaN,Berlin,NEIN,0,NaN


In [12]:
# and we merge the datasets of 2021 and 2023 based on all the features
comp_factor=pd.merge(df_2021_factor,df_2023_factor,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest',factor_to_check],suffixes=["_2021","_2023"])


In [13]:
comp_factor=comp_factor[comp_factor[factor_to_check]=='JA']
comp_factor

,Alter,Besiedelung,Geschlecht,OstWest,mc03,absolute_2021,percent_2021,absolute_2023,percent_2023
0,NaN,NaN,NaN,West,JA,3274,66.342452,3279,60.755976
1,NaN,NaN,Weiblich,NaN,JA,1917,58.767627,1853,52.716927
2,NaN,NaN,Männlich,NaN,JA,1819,61.933946,1946,59.041262
3,NaN,NaN,Weiblich,West,JA,1696,64.560335,1594,57.010014
5,NaN,dicht besiedelt,NaN,NaN,JA,1594,72.421627,1521,65.503876
...,...,...,...,...,...,...,...,...,...
118,U30,NaN,NaN,Ost,JA,39,37.500000,50,42.016807
119,Ü65,NaN,NaN,Berlin,JA,33,67.346939,46,59.740260
124,U30,NaN,NaN,Berlin,JA,19,47.500000,29,65.909091
126,NaN,mittlere Besiedlungsdichte,NaN,Berlin,JA,0,NaN,0,NaN


In [14]:
comp_factor=comp_factor[comp_factor.columns[comp_factor.columns!=factor_to_check]]

In [15]:
non_berlin_comp=comp_factor[comp_factor["OstWest"]!="Berlin"]
non_berlin_comp

,Alter,Besiedelung,Geschlecht,OstWest,absolute_2021,percent_2021,absolute_2023,percent_2023
0,NaN,NaN,NaN,West,3274,66.342452,3279,60.755976
1,NaN,NaN,Weiblich,NaN,1917,58.767627,1853,52.716927
2,NaN,NaN,Männlich,NaN,1819,61.933946,1946,59.041262
3,NaN,NaN,Weiblich,West,1696,64.560335,1594,57.010014
5,NaN,dicht besiedelt,NaN,NaN,1594,72.421627,1521,65.503876
6,NaN,NaN,Männlich,West,1578,68.370884,1685,64.782776
7,NaN,mittlere Besiedlungsdichte,NaN,NaN,1563,57.022984,1747,56.610499
8,NaN,mittlere Besiedlungsdichte,NaN,West,1448,62.819957,1600,61.967467
10,NaN,dicht besiedelt,NaN,West,1304,77.159763,1209,67.731092
12,30-49,NaN,NaN,NaN,1138,65.065752,1192,63.370548


In [16]:
non_berlin_comp["diff"]=non_berlin_comp["percent_2023"]-non_berlin_comp["percent_2021"]
non_berlin_comp=non_berlin_comp.sort_values(by="diff",ascending=False)
non_berlin_comp

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_59960/3213792933.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_berlin_comp["diff"]=non_berlin_comp["percent_2023"]-non_berlin_comp["percent_2021"]


,Alter,Besiedelung,Geschlecht,OstWest,absolute_2021,percent_2021,absolute_2023,percent_2023,diff
118,U30,NaN,NaN,Ost,39,37.500000,50,42.016807,4.516807
102,NaN,mittlere Besiedlungsdichte,NaN,Ost,115,26.376147,147,29.166667,2.790520
86,U30,mittlere Besiedlungsdichte,NaN,NaN,189,55.752212,243,57.995227,2.243014
114,Ü65,NaN,NaN,Ost,60,18.633540,80,20.618557,1.985016
79,U30,NaN,Männlich,NaN,258,55.010661,292,55.619048,0.608387
47,30-49,mittlere Besiedlungsdichte,NaN,NaN,468,61.660079,534,62.237762,0.577683
99,NaN,NaN,Männlich,Ost,141,28.313253,163,28.849558,0.536305
52,Ü65,mittlere Besiedlungsdichte,NaN,NaN,414,48.936170,460,49.250535,0.314365
23,NaN,mittlere Besiedlungsdichte,Weiblich,NaN,801,54.638472,877,54.607721,-0.030751
40,30-49,NaN,Weiblich,NaN,526,59.772727,568,59.663866,-0.108862


In [17]:
non_berlin_comp.head(30)

,Alter,Besiedelung,Geschlecht,OstWest,absolute_2021,percent_2021,absolute_2023,percent_2023,diff
118,U30,NaN,NaN,Ost,39,37.500000,50,42.016807,4.516807
102,NaN,mittlere Besiedlungsdichte,NaN,Ost,115,26.376147,147,29.166667,2.790520
86,U30,mittlere Besiedlungsdichte,NaN,NaN,189,55.752212,243,57.995227,2.243014
114,Ü65,NaN,NaN,Ost,60,18.633540,80,20.618557,1.985016
79,U30,NaN,Männlich,NaN,258,55.010661,292,55.619048,0.608387
47,30-49,mittlere Besiedlungsdichte,NaN,NaN,468,61.660079,534,62.237762,0.577683
99,NaN,NaN,Männlich,Ost,141,28.313253,163,28.849558,0.536305
52,Ü65,mittlere Besiedlungsdichte,NaN,NaN,414,48.936170,460,49.250535,0.314365
23,NaN,mittlere Besiedlungsdichte,Weiblich,NaN,801,54.638472,877,54.607721,-0.030751
40,30-49,NaN,Weiblich,NaN,526,59.772727,568,59.663866,-0.108862


In [18]:
non_berlin_comp.tail(30)

,Alter,Besiedelung,Geschlecht,OstWest,absolute_2021,percent_2021,absolute_2023,percent_2023,diff
48,50-64,dicht besiedelt,NaN,NaN,441,75.643225,445,72.475570,-3.167655
31,30-49,NaN,Männlich,NaN,612,70.425777,624,67.168999,-3.256778
44,50-64,mittlere Besiedlungsdichte,NaN,NaN,492,61.731493,510,58.285714,-3.445779
6,NaN,NaN,Männlich,West,1578,68.370884,1685,64.782776,-3.588108
35,50-64,NaN,Männlich,NaN,570,65.592635,589,61.675393,-3.917243
14,50-64,NaN,NaN,NaN,1113,63.202726,1128,58.841941,-4.360785
39,50-64,NaN,Weiblich,NaN,543,60.874439,539,56.029106,-4.845333
36,U30,NaN,NaN,NaN,557,59.956943,568,55.092144,-4.864799
15,50-64,NaN,NaN,West,986,70.378301,1000,65.402224,-4.976078
0,NaN,NaN,NaN,West,3274,66.342452,3279,60.755976,-5.586476


In [19]:
#also we do the same for party preferences
df_2021_party=analyze_factor_patterns(df_2021,"pv01")
df_2021_party

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_59960/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_59960/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_59960/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute,percent
0,30-49,NaN,NaN,NaN,AFD,137,6.135244
1,30-49,NaN,NaN,NaN,ANDERE PARTEI,114,5.105240
2,30-49,NaN,NaN,NaN,CDU-CSU,477,21.361397
3,30-49,NaN,NaN,NaN,DIE GRUENEN,743,33.273623
4,30-49,NaN,NaN,NaN,DIE LINKE,143,6.403941
...,...,...,...,...,...,...,...
19,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE GRUENEN,881,25.403691
20,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE LINKE,200,5.767013
21,NaN,mittlere Besiedlungsdichte,NaN,NaN,FDP,471,13.581315
22,NaN,mittlere Besiedlungsdichte,NaN,NaN,SPD,537,15.484429


In [20]:
df_2023_party=analyze_factor_patterns(df_2023,"pv01")
df_2023_party

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_59960/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_59960/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_59960/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute,percent
0,30-49,NaN,NaN,NaN,AFD,271,12.212708
1,30-49,NaN,NaN,NaN,ANDERE PARTEI,77,3.470032
2,30-49,NaN,NaN,NaN,CDU-CSU,426,19.197837
3,30-49,NaN,NaN,NaN,DIE GRUENEN,534,24.064894
4,30-49,NaN,NaN,NaN,DIE LINKE,138,6.219018
...,...,...,...,...,...,...,...
19,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE GRUENEN,711,19.921547
20,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE LINKE,158,4.427010
21,NaN,mittlere Besiedlungsdichte,NaN,NaN,FDP,381,10.675259
22,NaN,mittlere Besiedlungsdichte,NaN,NaN,SPD,721,20.201737


In [21]:
df_2023_party[df_2023_party.percent.isna()]

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute,percent
24,NaN,gering besiedelt,NaN,Berlin,AFD,0,NaN
25,NaN,gering besiedelt,NaN,Berlin,ANDERE PARTEI,0,NaN
26,NaN,gering besiedelt,NaN,Berlin,CDU-CSU,0,NaN
27,NaN,gering besiedelt,NaN,Berlin,DIE GRUENEN,0,NaN
28,NaN,gering besiedelt,NaN,Berlin,DIE LINKE,0,NaN
29,NaN,gering besiedelt,NaN,Berlin,FDP,0,NaN
30,NaN,gering besiedelt,NaN,Berlin,SPD,0,NaN
31,NaN,gering besiedelt,NaN,Berlin,WUERDE NICHT WAEHLEN,0,NaN
48,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AFD,0,NaN
49,NaN,mittlere Besiedlungsdichte,NaN,Berlin,ANDERE PARTEI,0,NaN


In [22]:
df_2021_party[df_2021_party.percent.isna()]

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute,percent
24,NaN,gering besiedelt,NaN,Berlin,AFD,0,NaN
25,NaN,gering besiedelt,NaN,Berlin,ANDERE PARTEI,0,NaN
26,NaN,gering besiedelt,NaN,Berlin,CDU-CSU,0,NaN
27,NaN,gering besiedelt,NaN,Berlin,DIE GRUENEN,0,NaN
28,NaN,gering besiedelt,NaN,Berlin,DIE LINKE,0,NaN
29,NaN,gering besiedelt,NaN,Berlin,FDP,0,NaN
30,NaN,gering besiedelt,NaN,Berlin,SPD,0,NaN
31,NaN,gering besiedelt,NaN,Berlin,WUERDE NICHT WAEHLEN,0,NaN
48,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AFD,0,NaN
49,NaN,mittlere Besiedlungsdichte,NaN,Berlin,ANDERE PARTEI,0,NaN


In [23]:
#we also merge the datasets of 2021 and 2023 based on all the features for the party preferences
comp_party=pd.merge(df_2021_party,df_2023_party,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', 'pv01'],suffixes=["_2021","_2023"])
comp_party["diff_percent"]=comp_party["percent_2023"]-comp_party["percent_2021"]
comp_party.sort_values("diff_percent")

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute_2021,percent_2021,absolute_2023,percent_2023,diff_percent
126,50-64,NaN,NaN,Berlin,SPD,19,29.230769,7,9.459459,-19.771310
157,U30,NaN,NaN,Ost,FDP,28,22.580645,5,3.759398,-18.821247
151,U30,NaN,NaN,Berlin,WUERDE NICHT WAEHLEN,9,18.000000,0,0.000000,-18.000000
170,Ü65,NaN,NaN,Berlin,CDU-CSU,30,35.294118,17,18.279570,-17.014548
146,U30,NaN,NaN,Berlin,CDU-CSU,8,16.000000,0,0.000000,-16.000000
...,...,...,...,...,...,...,...,...,...,...
475,NaN,mittlere Besiedlungsdichte,NaN,Berlin,DIE GRUENEN,0,NaN,0,NaN,NaN
476,NaN,mittlere Besiedlungsdichte,NaN,Berlin,DIE LINKE,0,NaN,0,NaN,NaN
477,NaN,mittlere Besiedlungsdichte,NaN,Berlin,FDP,0,NaN,0,NaN,NaN
478,NaN,mittlere Besiedlungsdichte,NaN,Berlin,SPD,0,NaN,0,NaN,NaN


In [24]:

#and we pivot it based on the party preferences
comp_party_pivot=comp_party.pivot(index=['Alter','Besiedelung','Geschlecht','OstWest'], columns='pv01')[['percent_2021','percent_2023',"absolute_2021","absolute_2023"]].reset_index()
comp_party_pivot

Alter                 Besiedelung Geschlecht OstWest percent_2021  \
pv01                                                               AFD   
0      NaN                         NaN        NaN  Berlin     2.950820   
1      NaN                         NaN        NaN     Ost    11.746522   
2      NaN                         NaN        NaN    West     4.289587   
3      NaN                         NaN   Männlich     NaN     7.808564   
4      NaN                         NaN   Männlich  Berlin     4.827586   
..     ...                         ...        ...     ...          ...   
60     Ü65                         NaN   Männlich     NaN     6.226766   
61     Ü65                         NaN   Weiblich     NaN     1.906275   
62     Ü65             dicht besiedelt        NaN     NaN     3.819918   
63     Ü65            gering besiedelt        NaN     NaN     5.040323   
64     Ü65  mittlere Besiedlungsdichte        NaN     NaN     3.435805   

                                                                 ...  \
pv01 ANDERE PARTEI    CDU-CSU DIE GRUENEN  DIE LINKE        FDP  ...   
0         3.606557  21.639344   26.885246   9.180328   7.213115  ...   
1         5.641422  23.106646   14.992272  12.905719  12.210201  ...   
2         3.412534  25.912933   29.835752   5.469622  11.880083  ...   
3         3.803526  24.055416   24.382872   7.506297  13.173804  ...   
4         2.068966  17.931034   27.586207   7.586207  12.413793  ...   
..             ...        ...         ...        ...        ...  ...   
60        0.743494  30.762082   14.962825   5.947955  13.661710  ...   
61        1.985703  36.219222   17.553614   6.592534  10.484512  ...   
62        1.227831  30.422920   17.462483   4.911323   8.867667  ...   
63        2.016129  35.483871   13.709677   8.266129  11.693548  ...   
64        1.265823  35.081374   16.817360   6.329114  14.104882  ...   

     absolute_2021                      absolute_2023                        \
pv01           SPD WUERDE NICHT WAEHLEN           AFD ANDERE PARTEI CDU-CSU   
0               60                   27            22             6      45   
1              159                   92           294            49     267   
2              961                  243           480           215    1609   
3              618                  147           526           156     940   
4               27                   13            15             3      20   
..             ...                  ...           ...           ...     ...   
60             267                   31            88             8     385   
61             247                   71            57            23     426   
62             202                   42            32             6     229   
63              96                   22            46            13     193   
64             216                   38            67            12     389   

                                                            
pv01 DIE GRUENEN DIE LINKE  FDP   SPD WUERDE NICHT WAEHLEN  
0             84        78   19    60                   23  
1            146       140   77   252                   70  
2           1518       240  641  1389                  318  
3            760       225  442   816                  214  
4             36        41    7    29                   12  
..           ...       ...  ...   ...                  ...  
60           154        54   92   321                   36  
61           219        41   94   388                   52  
62           101        37   84   292                   34  
63            65        24   22   117                   17  
64           207        34   80   300                   37  

[65 rows x 36 columns]

In [25]:
# to remove the double index ot the columns, we bring them to one name in one line
new_columns=[]
for index in comp_party_pivot:
    if index[1]!="":
        new_columns.append(index[1]+"_"+index[0])
    else:
        new_columns.append(index[0])
new_columns
comp_party_pivot.columns=new_columns
comp_party_pivot

,Alter,Besiedelung,Geschlecht,OstWest,AFD_percent_2021,ANDERE PARTEI_percent_2021,CDU-CSU_percent_2021,DIE GRUENEN_percent_2021,DIE LINKE_percent_2021,FDP_percent_2021,...,SPD_absolute_2021,WUERDE NICHT WAEHLEN_absolute_2021,AFD_absolute_2023,ANDERE PARTEI_absolute_2023,CDU-CSU_absolute_2023,DIE GRUENEN_absolute_2023,DIE LINKE_absolute_2023,FDP_absolute_2023,SPD_absolute_2023,WUERDE NICHT WAEHLEN_absolute_2023
0,NaN,NaN,NaN,Berlin,2.950820,3.606557,21.639344,26.885246,9.180328,7.213115,...,60,27,22,6,45,84,78,19,60,23
1,NaN,NaN,NaN,Ost,11.746522,5.641422,23.106646,14.992272,12.905719,12.210201,...,159,92,294,49,267,146,140,77,252,70
2,NaN,NaN,NaN,West,4.289587,3.412534,25.912933,29.835752,5.469622,11.880083,...,961,243,480,215,1609,1518,240,641,1389,318
3,NaN,NaN,Männlich,NaN,7.808564,3.803526,24.055416,24.382872,7.506297,13.173804,...,618,147,526,156,940,760,225,442,816,214
4,NaN,NaN,Männlich,Berlin,4.827586,2.068966,17.931034,27.586207,7.586207,12.413793,...,27,13,15,3,20,36,41,7,29,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Ü65,NaN,Männlich,NaN,6.226766,0.743494,30.762082,14.962825,5.947955,13.661710,...,267,31,88,8,385,154,54,92,321,36
61,Ü65,NaN,Weiblich,NaN,1.906275,1.985703,36.219222,17.553614,6.592534,10.484512,...,247,71,57,23,426,219,41,94,388,52
62,Ü65,dicht besiedelt,NaN,NaN,3.819918,1.227831,30.422920,17.462483,4.911323,8.867667,...,202,42,32,6,229,101,37,84,292,34
63,Ü65,gering besiedelt,NaN,NaN,5.040323,2.016129,35.483871,13.709677,8.266129,11.693548,...,96,22,46,13,193,65,24,22,117,17


In [26]:
comp_factor

,Alter,Besiedelung,Geschlecht,OstWest,absolute_2021,percent_2021,absolute_2023,percent_2023
0,NaN,NaN,NaN,West,3274,66.342452,3279,60.755976
1,NaN,NaN,Weiblich,NaN,1917,58.767627,1853,52.716927
2,NaN,NaN,Männlich,NaN,1819,61.933946,1946,59.041262
3,NaN,NaN,Weiblich,West,1696,64.560335,1594,57.010014
5,NaN,dicht besiedelt,NaN,NaN,1594,72.421627,1521,65.503876
...,...,...,...,...,...,...,...,...
118,U30,NaN,NaN,Ost,39,37.500000,50,42.016807
119,Ü65,NaN,NaN,Berlin,33,67.346939,46,59.740260
124,U30,NaN,NaN,Berlin,19,47.500000,29,65.909091
126,NaN,mittlere Besiedlungsdichte,NaN,Berlin,0,NaN,0,NaN


In [27]:
#and then again we merge both datasets
comp_migration_party_pivot=pd.merge(comp_factor,comp_party_pivot,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest'])
comp_migration_party_pivot

,Alter,Besiedelung,Geschlecht,OstWest,absolute_2021,percent_2021,absolute_2023,percent_2023,AFD_percent_2021,ANDERE PARTEI_percent_2021,...,SPD_absolute_2021,WUERDE NICHT WAEHLEN_absolute_2021,AFD_absolute_2023,ANDERE PARTEI_absolute_2023,CDU-CSU_absolute_2023,DIE GRUENEN_absolute_2023,DIE LINKE_absolute_2023,FDP_absolute_2023,SPD_absolute_2023,WUERDE NICHT WAEHLEN_absolute_2023
0,NaN,NaN,NaN,West,3274,66.342452,3279,60.755976,4.289587,3.412534,...,961,243,480,215,1609,1518,240,641,1389,318
1,NaN,NaN,Weiblich,NaN,1917,58.767627,1853,52.716927,3.076923,3.769231,...,562,215,270,114,981,988,233,295,885,197
2,NaN,NaN,Männlich,NaN,1819,61.933946,1946,59.041262,7.808564,3.803526,...,618,147,526,156,940,760,225,442,816,214
3,NaN,NaN,Weiblich,West,1696,64.560335,1594,57.010014,2.173913,3.228900,...,454,158,158,87,824,860,122,251,708,163
4,NaN,dicht besiedelt,NaN,NaN,1594,72.421627,1521,65.503876,4.021739,3.079710,...,407,121,189,68,523,760,220,218,710,156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,U30,NaN,NaN,Ost,39,37.500000,50,42.016807,9.677419,10.483871,...,5,8,22,9,25,25,19,5,26,2
61,Ü65,NaN,NaN,Berlin,33,67.346939,46,59.740260,1.176471,0.000000,...,21,10,2,3,17,14,21,8,24,4
62,U30,NaN,NaN,Berlin,19,47.500000,29,65.909091,0.000000,0.000000,...,10,9,3,1,0,15,24,0,14,0
63,NaN,mittlere Besiedlungsdichte,NaN,Berlin,0,NaN,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [28]:
comp_migration_party_pivot.columns

Index(['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', 'absolute_2021',
       'percent_2021', 'absolute_2023', 'percent_2023', 'AFD_percent_2021',
       'ANDERE PARTEI_percent_2021', 'CDU-CSU_percent_2021',
       'DIE GRUENEN_percent_2021', 'DIE LINKE_percent_2021',
       'FDP_percent_2021', 'SPD_percent_2021',
       'WUERDE NICHT WAEHLEN_percent_2021', 'AFD_percent_2023',
       'ANDERE PARTEI_percent_2023', 'CDU-CSU_percent_2023',
       'DIE GRUENEN_percent_2023', 'DIE LINKE_percent_2023',
       'FDP_percent_2023', 'SPD_percent_2023',
       'WUERDE NICHT WAEHLEN_percent_2023', 'AFD_absolute_2021',
       'ANDERE PARTEI_absolute_2021', 'CDU-CSU_absolute_2021',
       'DIE GRUENEN_absolute_2021', 'DIE LINKE_absolute_2021',
       'FDP_absolute_2021', 'SPD_absolute_2021',
       'WUERDE NICHT WAEHLEN_absolute_2021', 'AFD_absolute_2023',
       'ANDERE PARTEI_absolute_2023', 'CDU-CSU_absolute_2023',
       'DIE GRUENEN_absolute_2023', 'DIE LINKE_absolute_2023',
       'FDP_absol

In [29]:
#we take in the data from the zensus that gives a share of the population for each subgroup
#but only with two features defined as the same time
zensus_two_level=pd.read_csv("json_none.csv")
zensus_two_level

,Geschlecht,Besiedelung,OstWest,Alter,Share
0,NaN,NaN,Ost,U30,1.675804
1,NaN,gering besiedelt,NaN,U30,2.543912
2,NaN,dicht besiedelt,Ost,NaN,3.853220
3,NaN,NaN,Ost,30-49,4.559863
4,NaN,NaN,Ost,50-64,4.751534
5,NaN,NaN,Ost,Ü65,5.482466
6,NaN,gering besiedelt,Ost,NaN,5.491600
7,NaN,gering besiedelt,NaN,30-49,5.723338
8,NaN,mittlere Besiedlungsdichte,NaN,U30,5.842770
9,NaN,dicht besiedelt,NaN,U30,6.005464


In [30]:
# we merge this
comp_migration_party_pivot_zensus=pd.merge(comp_migration_party_pivot,zensus_two_level,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest'])


In [31]:
#and we filter only for the percentage results  
perc_share_df=comp_migration_party_pivot_zensus[comp_migration_party_pivot_zensus.columns[~comp_migration_party_pivot_zensus.columns.str.contains("absolute")]]

In [32]:
perc_share_df

,Alter,Besiedelung,Geschlecht,OstWest,percent_2021,percent_2023,AFD_percent_2021,ANDERE PARTEI_percent_2021,CDU-CSU_percent_2021,DIE GRUENEN_percent_2021,...,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
0,NaN,NaN,NaN,West,66.342452,60.755976,4.289587,3.412534,25.912933,29.835752,...,3.874980,7.488300,3.354134,25.101404,23.681747,3.744150,10.000000,21.669267,4.960998,79.480028
1,NaN,NaN,Weiblich,NaN,58.767627,52.716927,3.076923,3.769231,26.538462,30.230769,...,5.512821,6.813020,2.876609,24.753974,24.930608,5.879384,7.443856,22.331567,4.970982,51.683982
2,NaN,NaN,Männlich,NaN,61.933946,59.041262,7.808564,3.803526,24.055416,24.382872,...,3.702771,12.895317,3.824467,23.044864,18.632018,5.516058,10.835989,20.004903,5.246384,48.259419
3,NaN,NaN,Weiblich,West,64.560335,57.010014,2.173913,3.228900,26.854220,33.024297,...,5.051151,4.979515,2.741885,25.969114,27.103687,3.844942,7.910495,22.313268,5.137094,41.086342
4,NaN,dicht besiedelt,NaN,NaN,72.421627,65.503876,4.021739,3.079710,19.637681,37.101449,...,4.384058,6.645570,2.390999,18.389592,26.722925,7.735584,7.665260,24.964838,5.485232,34.799334
5,NaN,NaN,Männlich,West,68.370884,64.782776,6.395164,3.595291,24.976137,26.662424,...,2.704423,9.947482,3.954279,24.250850,20.327464,3.645351,12.048193,21.037998,4.788384,38.393686
6,NaN,mittlere Besiedlungsdichte,NaN,NaN,57.022984,56.610499,5.478662,3.835063,26.614764,25.403691,...,3.835063,10.339031,4.006725,25.581395,19.921547,4.427010,10.675259,20.201737,4.847296,44.144116
7,NaN,mittlere Besiedlungsdichte,NaN,West,62.819957,61.967467,4.245122,3.183841,27.114002,27.559055,...,3.457720,8.461538,3.846154,26.120401,21.839465,3.311037,11.605351,20.033445,4.782609,37.019270
8,NaN,dicht besiedelt,NaN,West,77.159763,67.731092,3.619490,2.923434,19.489559,39.907193,...,3.387471,5.224898,2.317129,19.218537,28.123580,5.043162,8.496138,26.351658,5.224898,26.952408
9,30-49,NaN,NaN,NaN,65.065752,63.370548,6.135244,5.105240,21.361397,33.273623,...,4.612629,12.212708,3.470032,19.197837,24.064894,6.219018,10.184768,17.755746,6.894998,27.801470


In [33]:
perc_share_df=perc_share_df.round(1)
perc_share_df


,Alter,Besiedelung,Geschlecht,OstWest,percent_2021,percent_2023,AFD_percent_2021,ANDERE PARTEI_percent_2021,CDU-CSU_percent_2021,DIE GRUENEN_percent_2021,...,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
0,NaN,NaN,NaN,West,66.3,60.8,4.3,3.4,25.9,29.8,...,3.9,7.5,3.4,25.1,23.7,3.7,10.0,21.7,5.0,79.5
1,NaN,NaN,Weiblich,NaN,58.8,52.7,3.1,3.8,26.5,30.2,...,5.5,6.8,2.9,24.8,24.9,5.9,7.4,22.3,5.0,51.7
2,NaN,NaN,Männlich,NaN,61.9,59.0,7.8,3.8,24.1,24.4,...,3.7,12.9,3.8,23.0,18.6,5.5,10.8,20.0,5.2,48.3
3,NaN,NaN,Weiblich,West,64.6,57.0,2.2,3.2,26.9,33.0,...,5.1,5.0,2.7,26.0,27.1,3.8,7.9,22.3,5.1,41.1
4,NaN,dicht besiedelt,NaN,NaN,72.4,65.5,4.0,3.1,19.6,37.1,...,4.4,6.6,2.4,18.4,26.7,7.7,7.7,25.0,5.5,34.8
5,NaN,NaN,Männlich,West,68.4,64.8,6.4,3.6,25.0,26.7,...,2.7,9.9,4.0,24.3,20.3,3.6,12.0,21.0,4.8,38.4
6,NaN,mittlere Besiedlungsdichte,NaN,NaN,57.0,56.6,5.5,3.8,26.6,25.4,...,3.8,10.3,4.0,25.6,19.9,4.4,10.7,20.2,4.8,44.1
7,NaN,mittlere Besiedlungsdichte,NaN,West,62.8,62.0,4.2,3.2,27.1,27.6,...,3.5,8.5,3.8,26.1,21.8,3.3,11.6,20.0,4.8,37.0
8,NaN,dicht besiedelt,NaN,West,77.2,67.7,3.6,2.9,19.5,39.9,...,3.4,5.2,2.3,19.2,28.1,5.0,8.5,26.4,5.2,27.0
9,30-49,NaN,NaN,NaN,65.1,63.4,6.1,5.1,21.4,33.3,...,4.6,12.2,3.5,19.2,24.1,6.2,10.2,17.8,6.9,27.8


In [34]:
perc_share_df["Geschlecht"]=perc_share_df["Geschlecht"].str.replace("Männlich","Maennlich")
perc_share_df["Alter"]=perc_share_df["Alter"].str.replace("Ü65","65plus")
perc_share_df["Besiedelung"]=perc_share_df["Besiedelung"].str.replace("dicht besiedelt","Stadt")
perc_share_df["Besiedelung"]=perc_share_df["Besiedelung"].str.replace("gering besiedelt","Dorf")
perc_share_df["Besiedelung"]=perc_share_df["Besiedelung"].str.replace("mittlere Besiedlungsdichte","Kleinstadt")
perc_share_df

,Alter,Besiedelung,Geschlecht,OstWest,percent_2021,percent_2023,AFD_percent_2021,ANDERE PARTEI_percent_2021,CDU-CSU_percent_2021,DIE GRUENEN_percent_2021,...,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
0,NaN,NaN,NaN,West,66.3,60.8,4.3,3.4,25.9,29.8,...,3.9,7.5,3.4,25.1,23.7,3.7,10.0,21.7,5.0,79.5
1,NaN,NaN,Weiblich,NaN,58.8,52.7,3.1,3.8,26.5,30.2,...,5.5,6.8,2.9,24.8,24.9,5.9,7.4,22.3,5.0,51.7
2,NaN,NaN,Maennlich,NaN,61.9,59.0,7.8,3.8,24.1,24.4,...,3.7,12.9,3.8,23.0,18.6,5.5,10.8,20.0,5.2,48.3
3,NaN,NaN,Weiblich,West,64.6,57.0,2.2,3.2,26.9,33.0,...,5.1,5.0,2.7,26.0,27.1,3.8,7.9,22.3,5.1,41.1
4,NaN,Stadt,NaN,NaN,72.4,65.5,4.0,3.1,19.6,37.1,...,4.4,6.6,2.4,18.4,26.7,7.7,7.7,25.0,5.5,34.8
5,NaN,NaN,Maennlich,West,68.4,64.8,6.4,3.6,25.0,26.7,...,2.7,9.9,4.0,24.3,20.3,3.6,12.0,21.0,4.8,38.4
6,NaN,Kleinstadt,NaN,NaN,57.0,56.6,5.5,3.8,26.6,25.4,...,3.8,10.3,4.0,25.6,19.9,4.4,10.7,20.2,4.8,44.1
7,NaN,Kleinstadt,NaN,West,62.8,62.0,4.2,3.2,27.1,27.6,...,3.5,8.5,3.8,26.1,21.8,3.3,11.6,20.0,4.8,37.0
8,NaN,Stadt,NaN,West,77.2,67.7,3.6,2.9,19.5,39.9,...,3.4,5.2,2.3,19.2,28.1,5.0,8.5,26.4,5.2,27.0
9,30-49,NaN,NaN,NaN,65.1,63.4,6.1,5.1,21.4,33.3,...,4.6,12.2,3.5,19.2,24.1,6.2,10.2,17.8,6.9,27.8


In [35]:
#and we write this into a csv file
perc_share_df.to_csv("perc_share_df.csv")

In [36]:
perc_share_df.Besiedelung.unique()

array([nan, 'Stadt', 'Kleinstadt', 'Dorf'], dtype=object)

In [37]:
json_array=[]
for index, row in perc_share_df.iterrows():
    print(row)
    base_dict=row[["Alter","Besiedelung","Geschlecht","OstWest","Share"]].to_dict()
    base_dict["kontakt"]=row[[ 'percent_2021', 'percent_2023']].to_dict()
    base_dict["wahl"]=row[['AFD_percent_2021', 'ANDERE PARTEI_percent_2021',
        'CDU-CSU_percent_2021', 'DIE GRUENEN_percent_2021',
        'DIE LINKE_percent_2021', 'FDP_percent_2021', 'SPD_percent_2021',
        'WUERDE NICHT WAEHLEN_percent_2021', 'AFD_percent_2023',
        'ANDERE PARTEI_percent_2023', 'CDU-CSU_percent_2023',
        'DIE GRUENEN_percent_2023', 'DIE LINKE_percent_2023',
        'FDP_percent_2023', 'SPD_percent_2023',
        'WUERDE NICHT WAEHLEN_percent_2023']].to_dict()
    json_array.append(base_dict)
json_array[0]

Alter                                 NaN
Besiedelung                           NaN
Geschlecht                            NaN
OstWest                              West
percent_2021                         66.3
percent_2023                         60.8
AFD_percent_2021                      4.3
ANDERE PARTEI_percent_2021            3.4
CDU-CSU_percent_2021                 25.9
DIE GRUENEN_percent_2021             29.8
DIE LINKE_percent_2021                5.5
FDP_percent_2021                     11.9
SPD_percent_2021                     15.3
WUERDE NICHT WAEHLEN_percent_2021     3.9
AFD_percent_2023                      7.5
ANDERE PARTEI_percent_2023            3.4
CDU-CSU_percent_2023                 25.1
DIE GRUENEN_percent_2023             23.7
DIE LINKE_percent_2023                3.7
FDP_percent_2023                     10.0
SPD_percent_2023                     21.7
WUERDE NICHT WAEHLEN_percent_2023     5.0
Share                                79.5
Name: 0, dtype: object
Alter      

{'Alter': nan,
 'Besiedelung': nan,
 'Geschlecht': nan,
 'OstWest': 'West',
 'Share': 79.5,
 'kontakt': {'percent_2021': 66.3, 'percent_2023': 60.8},
 'wahl': {'AFD_percent_2021': 4.3,
  'ANDERE PARTEI_percent_2021': 3.4,
  'CDU-CSU_percent_2021': 25.9,
  'DIE GRUENEN_percent_2021': 29.8,
  'DIE LINKE_percent_2021': 5.5,
  'FDP_percent_2021': 11.9,
  'SPD_percent_2021': 15.3,
  'WUERDE NICHT WAEHLEN_percent_2021': 3.9,
  'AFD_percent_2023': 7.5,
  'ANDERE PARTEI_percent_2023': 3.4,
  'CDU-CSU_percent_2023': 25.1,
  'DIE GRUENEN_percent_2023': 23.7,
  'DIE LINKE_percent_2023': 3.7,
  'FDP_percent_2023': 10.0,
  'SPD_percent_2023': 21.7,
  'WUERDE NICHT WAEHLEN_percent_2023': 5.0}}

In [38]:
! pip3 install simplejson


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [39]:
import simplejson as json
#And then we write out the file as a json file
with open('data_for_viz_kontakt_auslaender_1.json', 'w', encoding='utf-8') as file:
    json.dump(json_array, file, ignore_nan=True, indent=2)


In [40]:
comp_no_berlin_party=comp_party[comp_party["OstWest"]!="Berlin"].sort_values(by=["diff_percent"])
comp_no_berlin_party.head(60)

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute_2021,percent_2021,absolute_2023,percent_2023,diff_percent
157,U30,NaN,NaN,Ost,FDP,28,22.580645,5,3.759398,-18.821247
195,30-49,dicht besiedelt,NaN,NaN,DIE GRUENEN,428,49.365629,292,34.152047,-15.213582
35,30-49,NaN,Männlich,NaN,DIE GRUENEN,354,31.355182,209,17.666948,-13.688233
235,50-64,mittlere Besiedlungsdichte,NaN,NaN,DIE GRUENEN,325,32.146390,194,18.671800,-13.474590
243,U30,dicht besiedelt,NaN,NaN,DIE GRUENEN,262,50.384615,183,37.500000,-12.884615
355,NaN,dicht besiedelt,Männlich,NaN,DIE GRUENEN,493,36.437546,337,24.123121,-12.314425
59,50-64,NaN,Weiblich,NaN,DIE GRUENEN,346,34.088670,237,21.964782,-12.123888
443,NaN,dicht besiedelt,NaN,West,DIE GRUENEN,860,39.907193,619,28.123580,-11.783612
249,U30,gering besiedelt,NaN,NaN,ANDERE PARTEI,24,12.371134,3,1.612903,-10.758231
499,NaN,dicht besiedelt,NaN,NaN,DIE GRUENEN,1024,37.101449,760,26.722925,-10.378524


In [41]:
comp_no_berlin_party.tail(60)

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute_2021,percent_2021,absolute_2023,percent_2023,diff_percent
392,NaN,mittlere Besiedlungsdichte,Weiblich,NaN,AFD,43,2.489867,135,7.674815,5.184948
250,U30,gering besiedelt,NaN,NaN,CDU-CSU,28,14.432990,37,19.892473,5.459483
46,30-49,NaN,Weiblich,NaN,SPD,119,10.778986,169,16.312741,5.533756
136,50-64,NaN,NaN,West,AFD,72,4.183614,175,9.754738,5.571124
32,30-49,NaN,Männlich,NaN,AFD,102,9.034544,173,14.623838,5.589294
223,50-64,dicht besiedelt,NaN,NaN,WUERDE NICHT WAEHLEN,28,4.375000,69,10.058309,5.683309
216,50-64,dicht besiedelt,NaN,NaN,AFD,41,6.406250,83,12.099125,5.692875
190,Ü65,NaN,NaN,West,SPD,411,22.833333,550,28.810896,5.977562
208,30-49,mittlere Besiedlungsdichte,NaN,NaN,AFD,68,7.456140,126,13.519313,6.063173
0,30-49,NaN,NaN,NaN,AFD,137,6.135244,271,12.212708,6.077464


In [42]:
comp_factor
comp_factor["diff_percent"]=comp_factor["percent_2023"]-comp_factor["percent_2021"]
comp_no_berlin_migration=comp_factor[comp_factor["OstWest"]!="Berlin"]

comp_no_berlin_migration=comp_no_berlin_migration.sort_values("diff_percent")

In [43]:
comp_no_berlin_migration.head(20)

,Alter,Besiedelung,Geschlecht,OstWest,absolute_2021,percent_2021,absolute_2023,percent_2023,diff_percent
56,Ü65,dicht besiedelt,NaN,NaN,373,64.756944,313,50.160256,-14.596688
22,NaN,dicht besiedelt,Weiblich,NaN,832,71.972318,698,58.803707,-13.168612
110,U30,gering besiedelt,NaN,NaN,77,43.016760,55,30.726257,-12.290503
70,NaN,gering besiedelt,Männlich,NaN,295,47.811994,253,37.151248,-10.660745
69,U30,NaN,Weiblich,NaN,299,65.000000,276,54.545455,-10.454545
38,Ü65,NaN,Weiblich,NaN,549,53.300971,470,42.922374,-10.378596
41,NaN,gering besiedelt,NaN,West,522,55.531915,470,45.631068,-9.900847
10,NaN,dicht besiedelt,NaN,West,1304,77.159763,1209,67.731092,-9.428671
98,Ü65,gering besiedelt,NaN,NaN,141,41.715976,138,32.547170,-9.168807
71,U30,dicht besiedelt,NaN,NaN,291,70.802920,270,62.355658,-8.447262


In [44]:
comp_no_berlin_migration.tail(20)

,Alter,Besiedelung,Geschlecht,OstWest,absolute_2021,percent_2021,absolute_2023,percent_2023,diff_percent
45,30-49,dicht besiedelt,NaN,NaN,489,77.496038,493,75.729647,-1.766391
12,30-49,NaN,NaN,NaN,1138,65.065752,1192,63.370548,-1.695204
115,NaN,gering besiedelt,NaN,Ost,57,17.981073,61,16.353887,-1.627185
25,NaN,mittlere Besiedlungsdichte,Männlich,NaN,762,59.764706,870,58.783784,-0.980922
8,NaN,mittlere Besiedlungsdichte,NaN,West,1448,62.819957,1600,61.967467,-0.852490
95,NaN,NaN,Weiblich,Ost,142,28.063241,158,27.288428,-0.774813
112,50-64,NaN,NaN,Ost,77,26.643599,85,25.914634,-0.728964
7,NaN,mittlere Besiedlungsdichte,NaN,NaN,1563,57.022984,1747,56.610499,-0.412485
24,NaN,dicht besiedelt,Männlich,NaN,762,72.918660,823,72.511013,-0.407647
76,NaN,NaN,NaN,Ost,283,28.187251,321,28.059441,-0.127810
